In [2]:
import matplotlib.pyplot as plt
from astropy.coordinates import ICRS, Galactic, GalacticLSR, Galactocentric,SkyCoord
import astropy.units as u
import astropy.coordinates as apycoords
import pandas as pd
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})
import seaborn as sns
import statistics as stat
from matplotlib.figure import figaspect
import scipy
import numpy as np
import scipy.stats
import scipy.ndimage as ndimage
import matplotlib.pyplot as plt


# Preparing OB stars dataset for analysis

In [3]:
# Open the data file from Pantaleoni González et al. 2021 (10.1093/mnras/stab688) and read the header 
header = open('Data/als_ii_f.dat', 'r').readline()  

# Identify column breakpoints based on spaces in the header  
end_points = [0]  
prev_char = ''  
for i in range(len(header)):  
    if header[i] == ' ' and prev_char != ' ':  # Detect transition from non-space to space  
        end_points.append(i)  
    prev_char = header[i]  
end_points.append(len(header) - 1)  # Add the final column boundary  

# Create column intervals for fixed-width file parsing  
intervals = [[end_points[i], end_points[i + 1]] for i in range(len(end_points) - 1)]  

# Read the fixed-width formatted data into a Pandas DataFrame  
df = pd.read_fwf('Data/als_ii_f.dat', colspecs=intervals)  


In [4]:
df

,Cat,ID_ALS,ID_DR2,ID_GOSC,RA_ALS,DEC_ALS,RA_DR2,DEC_DR2,GLON,GLAT,...,BJ_dist_low,BJ_dist_high,SH_dist,SH_dist_low,SH_dist_high,SpT_ALS,Sflag,Pflag,Other_Crossmatch,Comments
0,M,1,3400904642850801792,NaN,05:31:14.8,+19:03:47.0,05:31:15.065,+19:03:49.818,186.65276,-8.01369,...,1278,1519,1332.0,1208.0,1447.0,NaN,S,V,NaN,NaN
1,M,2,3398880716823578368,NaN,05:50:08.7,+19:10:11.0,05:50:08.446,+19:09:58.003,188.90981,-4.15571,...,1655,2058,1747.0,1580.0,1944.0,A9II,S,V,NaN,NaN
2,M,3,3398822751945120128,NaN,05:52:22.3,+19:08:37.0,05:52:22.818,+19:08:58.923,189.19315,-3.70841,...,2833,4275,2750.0,2415.0,3396.0,"A0Ib, A0Iab",S,V,NaN,NaN
3,M,4,3350097928520768640,NaN,05:53:33.9,+17:46:31.0,05:53:33.898,+17:46:30.536,190.52464,-4.15982,...,1584,1844,NaN,NaN,NaN,NaN,S,V,NaN,NaN
4,M,5,3398557735283620224,NaN,05:54:37.8,+18:38:13.0,05:54:37.936,+18:38:13.599,189.90585,-3.50710,...,3922,6154,6430.0,4545.0,9480.0,NaN,S,P,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15657,I,20143,4146600472558819200,NaN,18:18:55.9,-13:46:53.0,18:18:55.833,-13:46:54.087,16.99104,0.77725,...,1373,1654,1602.0,1335.0,2064.0,B9III,S,V,NaN,NaN
15658,I,20145,4146598818993604480,NaN,18:18:46.2,-13:49:23.0,18:18:46.133,-13:49:23.464,16.93591,0.79221,...,1536,1775,1560.0,1420.0,1730.0,B5III,S,V,NaN,NaN
15659,I,20147,4146600399547025280,NaN,18:18:49.2,-13:48:04.0,18:18:49.168,-13:48:04.256,16.96110,0.79180,...,1591,1742,1601.0,1499.0,1719.0,B5III,S,V,NaN,NaN
15660,M,20151,5940106793062992128,NaN,16:47:03.6,-45:50:13.0,16:47:03.625,-45:50:14.423,339.55933,-0.39425,...,1468,3250,NaN,NaN,NaN,B5Ia+,S,P,NaN,Cluster membership: Westerlund 1.


In [5]:
df.keys()

Index(['Cat', 'ID_ALS', 'ID_DR2', 'ID_GOSC', 'RA_ALS', 'DEC_ALS', 'RA_DR2',
       'DEC_DR2', 'GLON', 'GLAT', 'Vmag', 'Gmag', 'Gmag_cor', 'BPmag', 'RPmag',
       'G_abs_mode', 'G_abs_mode_err', 'G_abs_P50', 'G_abs_P50_err',
       'BPmag-Gmag_cor', 'Gmag_cor-RPmag', 'CCDIST', 'PM_RA', 'PM_RA_err',
       'PM_DEC', 'PM_DEC_err', 'Plx', 'Plx_err', 'Sep_astr', 'RUWE',
       'ALS2_dist_mode', 'ALS2_dist_P50', 'ALS2_dist_mean', 'ALS2_dist_HDIl',
       'ALS2_dist_HDIh', 'ALS2_dist_P16', 'ALS2_dist_P84', 'BJ_dist',
       'BJ_dist_low', 'BJ_dist_high', 'SH_dist', 'SH_dist_low', 'SH_dist_high',
       'SpT_ALS', 'Sflag', 'Pflag', 'Other_Crossmatch', 'Comments'],
      dtype='object')

In [6]:
# Filter the DataFrame 'df' to select only OB stars based on specific criteria

ob_stars = df.loc[
    (df['Cat'] == 'M') &     # Select only rows where the 'Cat' (Category) column is 'M'. These are the OB stars.
    (df['GLAT'] >= -20.) &   # Include only stars with Galactic latitude (GLAT) between -20 and 20 degrees
    (df['GLAT'] <= 20.) &
    (df['Plx']/df['Plx_err'] >5) # Select stars with high quality distances
]


In [7]:
# Extract relevant columns from the 'ob_stars' DataFrame

# Galactic longitude and latitude (in degrees)
l = ob_stars.GLON.values  
b = ob_stars.GLAT.values  

# Parallax and its associated error (in milliarcseconds)
parallax = ob_stars.Plx.values  
parallax_error = ob_stars.Plx_err.values  

# Estimated distance from the ALS2 catalog (converted to kiloparsecs)
distance = ob_stars.ALS2_dist_P50.values / 1000  

# Source ID from Gaia DR2 catalog
source_id = ob_stars.ID_DR2.values  

# Proper motion in right ascension and declination (in mas/yr)
pmra = ob_stars.PM_RA.values  
pmdec = ob_stars.PM_DEC.values  


In [8]:
from astropy.coordinates import SkyCoord
import astropy.units as u

# Convert the l and b coordinates to RA and DEC in degrees
gc = SkyCoord(l=l*u.deg, b=b*u.deg, frame='galactic')
ra=gc.icrs.ra.value
dec=gc.icrs.dec.value

# Create a SkyCoord object with proper motions in ICRS frame
sc = SkyCoord(
    ra=ra * u.deg, dec=dec * u.deg, 
    pm_ra_cosdec=pmra * u.mas/u.yr, pm_dec=pmdec * u.mas/u.yr
)

# Convert proper motions from Equatorial (RA/Dec) to Galactic coordinates
pm_b = sc.galactic.pm_b.to(u.mas/u.yr).value  # Proper motion in Galactic latitude
pm_l = sc.galactic.pm_l_cosb.to(u.mas/u.yr).value  # Proper motion in Galactic longitude (cos(b) corrected)

# Convert Galactic coordinates (l, b) and distance to cartesian coordinates

c= apycoords.SkyCoord(l*u.deg,b*u.deg,distance=distance*u.kpc,frame='galactic')
gal=c.transform_to(GalacticLSR)
gal.representation_type = 'cartesian'
x = gal.x.to(u.kpc).value
y = gal.y.to(u.kpc).value
z = gal.z.to(u.kpc).value

# Convert Sky coordinates to Galactocentric coordinates in the cartesian coordinate system to obtain R

v_sun = apycoords.CartesianDifferential([11.1,245.,7.25]*u.km/u.s)
gc_frame= apycoords.Galactocentric(galcen_distance=8.34*u.kpc, galcen_v_sun=v_sun, z_sun=27.*u.pc)
gc = c.transform_to(gc_frame)
gc.representation_type= 'cylindrical'
R=gc.rho.to(u.kpc).value


In [9]:
import numpy as np

# Constants (all in km/s)
u_o = 11.1  
v_o = 12.24
w_o = 7.25
rho_o = 8.15
v_lsr = 229

# Compute trigonometric values
cos_l = np.cos(np.radians(l))
sin_l = np.sin(np.radians(l))
tan_b = np.tan(np.radians(b))
cos_b = np.cos(np.radians(b))

# Compute s_o for all elements at once
s_o = (u_o * cos_l) + (v_o * sin_l)

# Compute v_phi and s using vectorized operations
v_phi = v_lsr - (3 * (R - rho_o))
s = (v_phi * (rho_o / R) - v_lsr) * sin_l

# Compute vz_prime using vectorized expressions
vz_prime = (4.74 * distance * pm_b) / cos_b + w_o + ((s - s_o) * tan_b)


In [10]:
len(R[R<5]) # There are 22 stars with R< 5kpc. We need to remove them as 
            # the rotation curve in the above calculation isn't valid for R<5 kpc

22

In [11]:
# Select stars with R>5 kpc

source_id=source_id[R>5]
l=l[R>5]
b=b[R>5]
distance=distance[R>5]
x=x[R>5]
y=y[R>5]
z=z[R>5]
vz_prime=vz_prime[R>5]
pm_b=pm_b[R>5]
pm_l=pm_l[R>5]
R=R[R>5]

In [12]:
# Write the cleaned dataset with calculated and relevant values to a csv file 
import pandas
d = { 'source_id':source_id,'l': l, 'b': b,'distance':distance,'X':x,'Y':y,'Z':z,
     'Vzprime':vz_prime,'R':R,'pm_b':pm_b,'pm_l':pm_l}
dumm = pd.DataFrame(data=d)
dumm.to_csv('Cleaned_Data/OB_Stars_cleaned.csv', index = False)

# Prepare Upper Main Sequence stars (UMS) for analysis 

In [98]:
# Open the data file from Poggio et al. 2021 (10.1093/mnrasl/sly148)
# This dataset already contains the approximated vertical velocities Vz_prime.

df =  pd.read_csv('Data/UMS_sample_Added_Nearby_stars_updated.csv')

In [99]:
df

,source_id,l,b,d_new,Vzprime_new
0,5337774708293412992,290.711000,0.469779,2.917051,1.089035
1,5338153696217796224,289.556000,-0.025707,2.490043,-1.535947
2,5869589997125006976,307.311000,2.570170,1.285540,2.392665
3,5865533658171445120,309.647000,-0.065046,3.452518,-5.256970
4,6057519967635244800,298.766000,0.404107,4.008398,-3.939979
...,...,...,...,...,...
606972,2914535635649081600,230.677695,-22.360981,2.777019,37.251013
606973,2917256037872932480,228.076161,-21.779930,0.757466,-8.978425
606974,2917369150133151616,227.761588,-21.563179,2.493071,-17.806057
606975,2916425219398502528,228.848414,-21.568744,0.755457,-3.653782


In [100]:
df.keys()

Index(['source_id', 'l', 'b', 'd_new', 'Vzprime_new'], dtype='object')

In [101]:
#Compare with OB stars and filter out the common stars since the UMS sample also includes some OB stars. 
df= df[~df.source_id.isin(ob_stars.ID_DR2.values)]

In [102]:

ums_stars = df.loc[     
    (df['b'] >= -20.) &   # Include only stars with Galactic latitude (GLAT) between -20 and 20 degrees
    (df['b'] <= 20.) ]

# (df['parallax']/df['parallax_error'] >5) & # Select stars with high quality distances


In [103]:
# Extract relevant columns from the 'ums_stars' DataFrame

# Galactic longitude and latitude (in degrees)

l= ums_stars.l.values 
b= ums_stars.b.values

# Estimated distance from the UMS catalog in kpc

distance= ums_stars.d_new.values

#Source ID of stars in the Gaia catalogue

source_id=ums_stars.source_id.values

# Approximated vertical velocity of stars vz_prime
vz_prime=ums_stars.Vzprime_new.values 


# Convert Sky coordinates to Galactic coordinates in the cartesian coordinate system

c= apycoords.SkyCoord(l*u.deg,b*u.deg,distance=distance*u.kpc,frame='galactic')
gal=c.transform_to(GalacticLSR)
gal.representation_type = 'cartesian'
x = gal.x.to(u.kpc).value
y = gal.y.to(u.kpc).value
z = gal.z.to(u.kpc).value


v_sun = apycoords.CartesianDifferential([11.1,245.,7.25]*u.km/u.s)
gc_frame= apycoords.Galactocentric(galcen_distance=8.34*u.kpc, galcen_v_sun=v_sun, z_sun=27.*u.pc)
gc = c.transform_to(gc_frame)
gc.representation_type = 'cylindrical'
gc.differential_type= 'cylindrical'

R=gc.rho.to(u.kpc).value

In [104]:
source_id=source_id[R>5]
l=l[R>5]
b=b[R>5]
distance=distance[R>5]
x=x[R>5]
y=y[R>5]
z=z[R>5]
vz_prime=vz_prime[R>5]
R=R[R>5]

In [106]:
# Write the cleaned dataset with calculated and relevant values to a csv file 

import pandas
d = { 'source_id':source_id,'l': l, 'b': b,'distance':distance,'X':x,'Y':y,'Z':z,
     'Vzprime':vz_prime,'R':R}
dumm = pd.DataFrame(data=d)
dumm.to_csv('Cleaned_Data/UMS_Stars_cleaned.csv', index = False)

# Prepare Open Clusters for analysis 

In [20]:
# Open the data file from Cantat-Gaudin et al.2020 (10.1051/0004-6361/202038192)

from astropy.io.votable import parse_single_table
def votable_to_pandas(path):
    table = parse_single_table(path).to_table()    
    return table.to_pandas()

df= votable_to_pandas("Data/open_cluster_table.vot")

In [21]:
df

,Cluster,RA_ICRS,DE_ICRS,GLON,GLAT,r50,nbstars07,pmRA_,e_pmRA_,pmDE,...,AVNN,DMNN,DistPc,X,Y,Z,Rgc,SimbadName,_RA.icrs,_DE.icrs
0,ASCC_10,51.869999,34.980999,155.723007,-17.770,0.558,65,-1.737,0.159,-1.368,...,0.63,9.10,660.0,-573.0,258.0,-201.0,8917.0,[KPR2005] 10,51.870,34.981
1,ASCC_101,288.398987,36.368999,68.028000,11.608,0.372,69,0.934,0.205,1.288,...,0.19,8.08,412.0,151.0,374.0,83.0,8197.0,[KPR2005] 101,288.399,36.369
2,ASCC_105,295.548004,27.365999,62.825001,2.063,0.648,113,1.464,0.162,-1.635,...,0.41,8.77,567.0,259.0,504.0,20.0,8096.0,[KPR2005] 105,295.548,27.366
3,ASCC_107,297.164001,21.987000,58.903999,-1.901,0.174,57,-0.155,0.101,-5.156,...,1.69,9.70,870.0,449.0,744.0,-28.0,7925.0,[KPR2005] 107,297.164,21.987
4,ASCC_108,298.306000,39.348999,74.377998,6.074,0.537,188,-0.519,0.099,-1.690,...,0.34,10.32,1160.0,310.0,1110.0,122.0,8105.0,[KPR2005] 108,298.306,39.349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012,vdBergh_1,99.275002,3.078000,208.556000,-1.781,0.030,68,0.396,0.180,-0.771,...,1.89,11.45,1946.0,-1709.0,-930.0,-60.0,10092.0,Cl VDB 1,99.275,3.078
2013,vdBergh_80,97.738998,-9.625000,219.246994,-8.891,0.143,75,-3.298,0.397,0.418,...,1.53,9.80,910.0,-696.0,-569.0,-140.0,9054.0,Cl VDB 80,97.739,-9.625
2014,vdBergh_83,100.025002,-27.188999,236.447998,-14.329,0.151,48,-2.896,0.193,3.199,...,0.51,9.76,897.0,-480.0,-724.0,-222.0,8850.0,Cl VDB 83,100.025,-27.189
2015,vdBergh_85,101.718002,1.320000,211.237000,-0.414,0.040,29,-1.000,0.155,0.328,...,1.35,11.14,1690.0,-1445.0,-876.0,-12.0,9824.0,Cl VDB 85,101.718,1.320


In [22]:
df.keys()

Index(['Cluster', 'RA_ICRS', 'DE_ICRS', 'GLON', 'GLAT', 'r50', 'nbstars07',
       'pmRA_', 'e_pmRA_', 'pmDE', 'e_pmDE', 'plx', 'e_plx', 'Flag', 'AgeNN',
       'AVNN', 'DMNN', 'DistPc', 'X', 'Y', 'Z', 'Rgc', 'SimbadName',
       '_RA.icrs', '_DE.icrs'],
      dtype='object')

In [23]:

open_clusters = df.loc[     
    (df['GLAT'] >= -20.) &   # Include only stars with Galactic latitude (GLAT) between -20 and 20 degrees
    (df['GLAT'] <= 20.) &
    (df['plx']/df['e_plx'] >5)  # Select stars with high quality distances
]


In [24]:
# Extract relevant columns from the 'open_clusters' DataFrame

# Galactic longitude and latitude (in degrees)

l= open_clusters.GLON.values 
b= open_clusters.GLAT.values 

# Estimated distance from the UMS catalog in kpc

distance= open_clusters.DistPc.values/1000 

#Source ID of clusters in the Gaia catalogue

source_id=open_clusters.Cluster.values

# Right ascension and declination (J2000, in degrees)
ra = open_clusters.RA_ICRS.values  
dec = open_clusters.DE_ICRS.values  

# Proper motion in right ascension and declination (in mas/yr)
pmra = open_clusters.pmRA_.values  
pmdec = open_clusters.pmDE.values  

# Log10 of Ages of the clusters. This is important because we need to choose young clusters for our analysis.
LogAge=open_clusters.AgeNN.values

In [25]:

# Convert Sky coordinates to Galactic coordinates in the cartesian coordinate system

c= apycoords.SkyCoord(l*u.deg,b*u.deg,distance=distance*u.kpc,frame='galactic')
gal=c.transform_to(GalacticLSR)
gal.representation_type = 'cartesian'
x = gal.x.to(u.kpc).value
y = gal.y.to(u.kpc).value
z = gal.z.to(u.kpc).value

# Convert Sky coordinates to Galactocentric coordinates in the cartesian coordinate system to obtain R

v_sun = apycoords.CartesianDifferential([11.1,245.,7.25]*u.km/u.s)
gc_frame= apycoords.Galactocentric(galcen_distance=8.34*u.kpc, galcen_v_sun=v_sun, z_sun=27.*u.pc)
gc = c.transform_to(gc_frame)
gc.representation_type= 'cylindrical'
R=gc.rho.to(u.kpc).value

# Create a SkyCoord object with proper motions in ICRS frame
sc = SkyCoord(ra=ra * u.deg, dec=dec * u.deg, 
    pm_ra_cosdec=pmra * u.mas/u.yr, pm_dec=pmdec * u.mas/u.yr)

# Convert proper motions from Equatorial (RA/Dec) to Galactic coordinates
pm_b = sc.galactic.pm_b.to(u.mas/u.yr).value  # Proper motion in Galactic latitude
pm_l = sc.galactic.pm_l_cosb.to(u.mas/u.yr).value  # Proper motion in Galactic longitude (cos(b) corrected)


In [26]:
min(R) # Minimum R of open clusters is >5. So we don't have to apply a cut in the galactocentric distance R here. 

5.1372606300826105

In [27]:
import numpy as np

# Constants (all in km/s)
u_o = 11.1  
v_o = 12.24
w_o = 7.25
rho_o = 8.15
v_lsr = 229

# Compute trigonometric values once for efficiency
cos_l = np.cos(np.radians(l))
sin_l = np.sin(np.radians(l))
tan_b = np.tan(np.radians(b))
cos_b = np.cos(np.radians(b))

# Compute s_o for all elements at once
s_o = (u_o * cos_l) + (v_o * sin_l)

# Compute v_phi and s using vectorized operations
v_phi = v_lsr - (3 * (R - rho_o))
s = (v_phi * (rho_o / R) - v_lsr) * sin_l

# Compute vz_prime using vectorized expressions
vz_prime = (4.74 * distance * pm_b) / cos_b + w_o + ((s - s_o) * tan_b)


In [28]:
# Write the cleaned dataset with calculated and relevant values to a csv file 

import pandas
d = { 'source_id':source_id,'l': l, 'b': b,'distance':distance,'X':x,'Y':y,'Z':z,
     'Vzprime':vz_prime,'R':R,'pm_b':pm_b,'pm_l':pm_l}
dumm = pd.DataFrame(data=d)
dumm.to_csv('Cleaned_Data/Open_Clusters_cleaned.csv', index = False)

# Prepare the Giant sample (a.k.a our Control sample) for analysis 

In [108]:
# Open the data file from Poggio et al. 2021 (10.1093/mnrasl/sly148)
# This is a huge data file (4 GB). So won't be uploading this to Github. 
# However the cleaned dataset would be available.

df =  pd.read_csv('Data/Giants_EDR3_P21_VzprimeReid_and_Vz_EDR3workshop.csv')

In [109]:
df.keys()

Index(['source_id', 'ra', 'dec', 'parallax', 'parallax_error', 'pmra',
       'pmra_error', 'pmdec', 'pmdec_error', 'parallax_pmra_corr',
       'parallax_pmdec_corr', 'pmra_pmdec_corr', 'astrometric_excess_noise',
       'ruwe', 'astrometric_params_solved', 'pseudocolour', 'phot_g_mean_mag',
       'bp_rp', 'bp_g', 'g_rp', 'phot_bp_rp_excess_factor',
       'dr2_radial_velocity', 'dr2_radial_velocity_error', 'l', 'b', 'd',
       'mul', 'mub', 'Vzprime', 'R', 'Vz'],
      dtype='object')

In [110]:
df

,source_id,ra,dec,parallax,parallax_error,pmra,pmra_error,pmdec,pmdec_error,parallax_pmra_corr,...,dr2_radial_velocity,dr2_radial_velocity_error,l,b,d,mul,mub,Vzprime,R,Vz
0,5875917446037024896,230.385962,-60.002976,0.455928,0.016198,-4.609504,0.016957,-2.224678,0.016342,-0.017230,...,NaN,NaN,320.678407,-2.408556,2.121875,-5.08360,0.594713,15.488938,6.619709,NaN
1,5830872722357089408,248.681570,-60.002970,0.123893,0.017607,-3.656951,0.017865,-3.761535,0.016163,-0.100365,...,NaN,NaN,327.641943,-8.334150,6.721288,-5.24370,0.161288,30.430689,4.352059,NaN
2,6056055619971132800,195.397309,-60.002969,0.139415,0.035886,-6.957928,0.028133,-0.848039,0.034052,0.074424,...,NaN,NaN,304.201974,2.843746,5.903192,-6.98451,-0.590364,-10.724089,6.847880,NaN
3,5336273222098284032,174.416371,-60.002952,0.409276,0.015147,-10.777808,0.014980,1.747391,0.015158,-0.037215,...,NaN,NaN,293.827987,1.558375,2.353839,-10.83500,-1.348320,-7.777682,7.487464,NaN
4,5293431369939218304,113.209385,-60.002937,0.377853,0.008859,1.623126,0.011123,5.640391,0.010338,0.154692,...,121.75645,0.593471,271.720650,-18.358706,2.530645,-4.54335,3.715710,50.375100,8.400252,16.392027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12663055,5833502066998184832,236.985734,-60.617324,0.227357,0.025756,-6.544422,0.030026,-6.933191,0.029767,-0.121386,...,NaN,NaN,322.980245,-4.811991,4.153734,-9.42504,-1.437800,-13.993277,5.423682,NaN
12663056,5408437193835983104,151.821861,-46.823973,0.176428,0.014590,-4.780805,0.015464,1.846640,0.015051,0.104516,...,NaN,NaN,276.002155,7.284864,5.108778,-4.95550,-1.307350,-20.279530,9.112590,NaN
12663057,2065104049265345152,313.205727,40.329919,0.207124,0.020441,-3.491348,0.019453,-3.761593,0.019108,-0.177255,...,NaN,NaN,81.736383,-2.732317,4.482900,-5.12578,0.255779,14.318367,8.692710,NaN
12663058,2065087831468643712,313.498676,40.196645,0.385670,0.017386,-4.463765,0.017671,-6.486299,0.017747,-0.060343,...,NaN,NaN,81.777307,-2.989346,2.492330,-7.83539,-0.777093,-0.988435,8.147350,NaN


This dataset consists of older stars. That means most of the stars have full 3D velocities available. This means that we don't need to approximate the vertical velocities. The dataframe already contains Vzprime and Vz. Vz is the actual velocities and Vzprime is the approximated velocities. We can use this catalogue to check the accuracy of the approximation we use to find the vertical velocities of stars. We also will be using the actual vertical velocities for the giant stars for any further analysis. We save it under Vzprime for the ease of coding though.

In [111]:

giant_stars = df.loc[     
    (df['b'] >= -20.) &   # Include only stars with Galactic latitude (GLAT) between -20 and 20 degrees
    (df['b'] <= 20.) &
    (df['parallax']/df['parallax_error'] >5) & # Select stars with high quality distances
    df['Vz'].notnull() # select all the stars with a valid Vz. Since we know the actual Vz of most of the stars, 
                       # we can check the accuracy of Vz_prime with the actual Vz. 
]

# We dont need to apply the R>5 kpc cut because here we will be using actual vertical velocities not the approximation.

In [123]:
# Extract relevant columns from the 'ums_stars' DataFrame

# Galactic longitude and latitude (in degrees)

l= giant_stars.l.values
b= giant_stars.b.values 

# Estimated distance from the Giant stars catalog in kpc

distance= giant_stars.d.values

# Galactocentric distance R in kpc

R=giant_stars.R.values


#Source ID of stars in the Gaia catalogue

source_id=giant_stars.source_id.values

# Actual vertical velocity of stars vz

vz=giant_stars.Vz.values 

# Approximated vertical velocity of stars vz_prime

vz_prime=giant_stars.Vzprime.values 


# Proper motion in Galactic latitude
pm_b = giant_stars['mub'].values

# Proper motion in Galactic longitude (cos(b) corrected)
pm_l = giant_stars['mul'].values


# Convert Sky coordinates to Galactic coordinates in the cartesian coordinate system

c= apycoords.SkyCoord(l*u.deg,b*u.deg,distance*u.kpc,frame='galactic')
gal=c.transform_to(GalacticLSR)
gal.representation_type = 'cartesian'
x = gal.x.to(u.kpc).value
y = gal.y.to(u.kpc).value
z = gal.z.to(u.kpc).value


In [126]:
# Write the cleaned dataset with calculated and relevant values to a csv file 

import pandas
d = { 'source_id':source_id,'l': l, 'b': b,'distance':distance,'X':x,'Y':y,'Z':z,
     'Vzprime':vz,'R':R,'pm_b':pm_b,'pm_l':pm_l}
dumm = pd.DataFrame(data=d)
dumm.to_csv('Cleaned_Data/Giant_Stars_cleaned.csv', index = False)